<a href="https://colab.research.google.com/github/ThousandAI/Application-of-AI/blob/main/class02/class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Autograd

In [1]:
import torch
import torch.nn as nn

In [2]:
# grad = True
tensor = torch.rand(3, requires_grad=True)
print(tensor)
y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) # 2*tensor + 12

tensor([0.5174, 0.2284, 0.2127], requires_grad=True)
tensor([6.5174, 6.2284, 6.2127], grad_fn=<AddBackward0>)
tensor([129.4313, 118.3781, 117.7930], grad_fn=<AddBackward0>)
tensor(121.8675, grad_fn=<MeanBackward0>)
tensor([13.0349, 12.4568, 12.4254])


In [3]:
output = torch.tensor([[1.6,-0.8,3.7],[0.2,-3.2,-1.6]])
softmax = nn.Softmax(dim=0)
print(softmax(output))


tensor([[0.8022, 0.9168, 0.9950],
        [0.1978, 0.0832, 0.0050]])


In [5]:
# grad = False
tensor = torch.rand(3, requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean()
print(z)
z.backward() # dz/d(tensor)

tensor([0.0231, 0.3810, 0.5619], requires_grad=True)
tensor([6.0231, 6.3810, 6.5619], grad_fn=<AddBackward0>)
tensor([110.8335, 124.1521, 131.1747], grad_fn=<AddBackward0>)
tensor(122.0534, grad_fn=<MeanBackward0>)


In [13]:
# not scalar
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z.backward()

tensor([0.3700, 0.5800, 0.3300], requires_grad=True)
tensor([6.3700, 6.5800, 6.3300], grad_fn=<AddBackward0>)
tensor([123.7307, 131.8892, 122.2067], grad_fn=<AddBackward0>)


In [9]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach()
print(y)

tensor([0.8273, 0.3031, 0.4802], requires_grad=True)
tensor([0.8273, 0.3031, 0.4802])


In [10]:
# grad 
with torch.no_grad():
  y = tensor + 2
  print(y)

tensor([2.8273, 2.3031, 2.4802])


In [11]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [12]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


## toy example

In [15]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True)
w2 = torch.rand(3,3, requires_grad=True)
w3 = torch.rand(3,2, requires_grad=True)
w4 = torch.rand(2,1, requires_grad=True)
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 10

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        w3 -= learning_rate * w3.grad
        w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")

    

epoch 1: 
w1 = tensor([[0.0471, 0.1226, 0.5485],
        [0.8144, 0.3007, 0.8098]], requires_grad=True)
 w2 = tensor([[0.4648, 0.8491, 0.7860],
        [0.1026, 0.0064, 0.9860],
        [0.5388, 0.4235, 0.6420]], requires_grad=True)
 w3 = tensor([[0.1179, 0.4701],
        [0.2846, 0.5691],
        [0.3665, 0.7700]], requires_grad=True)
 w4 = tensor([[0.1492],
        [0.1742]], requires_grad=True), loss = 0.99751854
epoch 2: 
w1 = tensor([[0.0460, 0.1219, 0.5477],
        [0.8119, 0.2991, 0.8080]], requires_grad=True)
 w2 = tensor([[0.4642, 0.8482, 0.7848],
        [0.1023, 0.0060, 0.9855],
        [0.5380, 0.4224, 0.6404]], requires_grad=True)
 w3 = tensor([[0.1167, 0.4687],
        [0.2833, 0.5676],
        [0.3646, 0.7677]], requires_grad=True)
 w4 = tensor([[0.1410],
        [0.1554]], requires_grad=True), loss = 0.94593567
epoch 3: 
w1 = tensor([[0.0452, 0.1214, 0.5470],
        [0.8098, 0.2978, 0.8065]], requires_grad=True)
 w2 = tensor([[0.4636, 0.8474, 0.7838],
        [0.1021,

# Dataset & DataLoader

In [16]:
from torch.utils.data import Dataset, DataLoader

In [18]:
class ScoreDataset(Dataset):
  def __init__(self):
    self.x = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18]]) # 6x3
    self.y = torch.tensor([[1],[0],[1],[1],[1],[0]]) # 6x1

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.x.size(0)

In [19]:
score_dataset = ScoreDataset()

In [22]:
print(score_dataset.x)
print(score_dataset.y)

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[1],
        [0],
        [1],
        [1],
        [1],
        [0]])


In [21]:
dataLoader = DataLoader(dataset=score_dataset,batch_size=2,shuffle=True)

In [23]:
for i, (x,y) in enumerate(dataLoader):
  print(f"{i}: x={x}, y={y}")

0: x=tensor([[10, 11, 12],
        [13, 14, 15]]), y=tensor([[1],
        [1]])
1: x=tensor([[4, 5, 6],
        [7, 8, 9]]), y=tensor([[0],
        [1]])
2: x=tensor([[ 1,  2,  3],
        [16, 17, 18]]), y=tensor([[1],
        [0]])


In [31]:
from torch.utils.data import Dataset, DataLoader

class ScoreDataset(Dataset):
	def __init__(self):
		self.x = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18]])
		self.y = torch.tensor([[1],[0],[1],[1],[1],[0]])

	def __getitem__(self,index):
		return self.x[index], self.y[index]

	def __len__(self):
		return self.x.size(0)

score_dataset = ScoreDataset()

print(score_dataset.x)
print(score_dataset.y)

dataLoader = DataLoader(dataset=score_dataset,batch_size=2,shuffle=True)

for i, (x,y) in enumerate(dataLoader):
	print(f"{i}: x={x}, y={y}")

tensor([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12],
        [13, 14, 15],
        [16, 17, 18]])
tensor([[1],
        [0],
        [1],
        [1],
        [1],
        [0]])
0: x=tensor([[10, 11, 12],
        [13, 14, 15]]), y=tensor([[1],
        [1]])
1: x=tensor([[16, 17, 18],
        [ 4,  5,  6]]), y=tensor([[0],
        [0]])
2: x=tensor([[7, 8, 9],
        [1, 2, 3]]), y=tensor([[1],
        [1]])
